In [138]:
import pandas as pd


In [139]:
raw_mail_data = pd.read_csv("mail_data.csv")
#зареждаме файла с данни

In [140]:
raw_mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [141]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

In [142]:
raw_mail_data.isnull().sum()

Category    0
Message     0
dtype: int64

In [143]:
df = raw_mail_data.where((pd.notnull(raw_mail_data)), '')
#заменяме празните данни с ' '

In [144]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [145]:
df.shape

(5572, 2)

In [146]:
#заменяме ham с 1 и spam с 0
df['Category'] = df['Category'].map({'spam': 0, 'ham': 1})

In [147]:
df.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [148]:
df.loc[df['Category'] == 'spam', 'Category',] = 0
df.loc[df['Category'] == 'ham', 'Category',] = 1

In [149]:
df.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [150]:
#разделяме двете категории на отделни сетове
X = df['Message']
Y = df['Category']

In [151]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object

In [152]:
Y

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: int64

In [153]:
#сплитваме групите на тренировъчни и на тестови, като използваме 80% за тестовете
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

In [ ]:

feature_extraction = TfidfVectorizer(min_df = 1, stop_words="english", binary=True)
X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   int64 
 1   Message   5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [156]:
y_train = y_train.astype("int")
y_test = y_test.astype("int")

In [ ]:

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
model1 = RandomForestClassifier()
model2 = GradientBoostingClassifier()
model3 = AdaBoostClassifier()
model1.fit(X_train_features, y_train)
model2.fit(X_train_features, y_train)
model3.fit(X_train_features, y_train)

AdaBoostClassifier()

In [ ]:

prediction_train_data1 = model1.predict(X_train_features)
accuracy_train_data1 = accuracy_score(y_train, prediction_train_data1)
prediction_train_data2 = model2.predict(X_train_features)
accuracy_train_data2 = accuracy_score(y_train, prediction_train_data2)
prediction_train_data3 = model3.predict(X_train_features)
accuracy_train_data3 = accuracy_score(y_train, prediction_train_data3)

In [159]:
print("Accuracy on test data 1: ", accuracy_test_data1)
print("Accuracy on test data 2: ", accuracy_test_data2)
print("Accuracy on test data 3: ", accuracy_test_data3)

Accuracy on test data 1:  0.97847533632287
Accuracy on test data 2:  0.9641255605381166
Accuracy on test data 3:  0.9076233183856502


In [ ]:

import pickle
pickle.dump(model1, open("gradient_boosting_classifier.pkl", "wb"))
pickle.dump(model2, open("random_forest_classifier.pkl", "wb"))
pickle.dump(model3, open("ada_boost_classifier.pkl", "wb"))

pickle.dump(feature_extraction, open("feature_extraction.pkl", "wb"))

In [ ]:
from sklearn.linear_model import LogisticRegression

In [163]:
meta_model = LogisticRegression()

In [167]:
X_train_meta = pd.DataFrame({
    'model1': prediction_train_data1,
    'model2': prediction_train_data2,
    'model3': prediction_train_data3
})

In [168]:
meta_model.fit(X_train_meta, y_train)

LogisticRegression()

In [170]:
pred_test_model1 = model1.predict(X_test_features)
pred_test_model2 = model2.predict(X_test_features)
pred_test_model3 = model3.predict(X_test_features)

In [171]:
X_test_meta = pd.DataFrame({
    'model1': pred_test_model1,
    'model2': pred_test_model2,
    'model3': pred_test_model3
})

In [172]:
final_predictions = meta_model.predict(X_test_meta)

In [173]:
accuracy_test_data = accuracy_score(y_test, final_predictions)
print("Test Accuracy: ", accuracy_test_data)

Test Accuracy:  0.9802690582959641


In [174]:
pickle.dump(meta_model, open("meta_model.pkl", "wb"))